# Introduction

This Jupyter Notebook provides a comprehensive workflow for cleaning and preprocessing a dataset of YouTube trending videos. 

The dataset contains various attributes related to the videos, and can be found [here](https://www.kaggle.com/datasets/canerkonuk/youtube-trending-videos-global).

# Data Cleaning

- We start by importing the necessary libraries.

In [1]:
import pandas as pd
import re

- We read the CSV and handle the dates properly.

In [2]:
# Read data and convert data columns to appropriate data types
data = pd.read_csv('youtube_trending_videos_global.csv', parse_dates=['video_published_at', 'video_trending__date'])

# Convert the 'channel_published_at' column to datetime, with error handling for mixed formats
data['channel_published_at'] = pd.to_datetime(data['channel_published_at'], errors='coerce')

- We drop the columns that are not needed for the analysis.

In [3]:
columns_to_drop = [
    'video_description', 'video_title', 'video_tags', 'channel_title', 
    'video_default_thumbnail', 'channel_description', 'channel_custom_url', 
    'channel_localized_title', 'channel_localized_description'
]

data = data.drop(columns=columns_to_drop)

- Let's take a look at the null values.

In [4]:
data.isna().sum()

video_id                                0
video_published_at                      0
video_trending__date                    0
video_trending_country                  0
channel_id                              0
video_category_id                     767
video_duration                          0
video_dimension                         0
video_definition                        0
video_licensed_content                  0
video_view_count                       43
video_like_count                    12323
video_comment_count                  7558
channel_published_at               704584
channel_country                    168580
channel_view_count                      0
channel_subscriber_count                0
channel_have_hidden_subscribers         0
channel_video_count                     0
dtype: int64

- We fill the null values in `channel_country` with `"Unknown"` to facilitate the analysis.

In [5]:
data['channel_country'] = data['channel_country'].fillna('Unknown')

- We drop the rows with null values in key columns for the analysis.

In [6]:
data = data.dropna(subset=['video_category_id', 'video_view_count', 'video_like_count', 'video_comment_count'])

- Let's now check if there are any negative values in `video_view_count`, `video_like_count`, or `video_comment_count`.

In [7]:
negative_values = data[(data['video_view_count'] < 0) | (data['video_like_count'] < 0) | (data['video_comment_count'] < 0)]

negative_values

,video_id,video_published_at,video_trending__date,video_trending_country,channel_id,video_category_id,video_duration,video_dimension,video_definition,video_licensed_content,video_view_count,video_like_count,video_comment_count,channel_published_at,channel_country,channel_view_count,channel_subscriber_count,channel_have_hidden_subscribers,channel_video_count


- No negative values, so we don't need to remove any more rows.

- We convert `video_view_count`, `video_like_count`, and `video_comment_count` to integers.

In [8]:
data['video_view_count'] = data['video_view_count'].astype(int)
data['video_like_count'] = data['video_like_count'].astype(int)
data['video_comment_count'] = data['video_comment_count'].astype(int)

- We also want to convert the ISO 8601 duration to minutes, in order to facilitate the interpretation.

- We define a function for this.

In [9]:
def iso8601_to_minutes(duration):
    # Parse ISO 8601 duration using regex
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', duration)
    if not match:
        return None  # Handle invalid format
    
    hours, minutes, seconds = match.groups(default="0")
    
    # Convert to total minutes as a float
    total_minutes = int(hours) * 60 + int(minutes) + int(seconds) / 60
    return round(total_minutes, 2)  # Round to 2 decimal places

- Now we replace the `video_duration` column with the converted values.

In [10]:
data['video_duration'] = data['video_duration'].apply(iso8601_to_minutes)

- Let's save the cleaned data to a new CSV file.

In [11]:
data.to_csv('cleaned_youtube_trending_videos_global.csv', index=False)

data.head()

,video_id,video_published_at,video_trending__date,video_trending_country,channel_id,video_category_id,video_duration,video_dimension,video_definition,video_licensed_content,video_view_count,video_like_count,video_comment_count,channel_published_at,channel_country,channel_view_count,channel_subscriber_count,channel_have_hidden_subscribers,channel_video_count
0,bB3-CUMERIU,2024-10-11 00:00:06+00:00,2024-10-12,United Arab Emirates,UCNYi_zGmR519r5gYdOKLTjQ,Music,2.47,2d,hd,False,20535235,2042255,152933,2021-01-13 06:19:55.866890+00:00,Unknown,464615150,11600000,False,43
2,zfb0whgBBA8,2024-10-11 11:07:25+00:00,2024-10-12,United Arab Emirates,UCgGYPnVJytkr6sVNLQ-l0zQ,Gaming,43.40,2d,hd,True,853167,101155,10541,NaT,Saudi Arabia,114331110,1380000,False,314
4,UVb6QOKy0bI,2024-10-09 12:30:27+00:00,2024-10-12,United Arab Emirates,UCOzubmwpVZI7gD0Jf7Bk3Aw,Film & Animation,2.20,2d,hd,True,1730189,67522,2869,NaT,Unknown,19991522,40600,False,56
5,IINqHUM71Xo,2024-10-12 02:03:10+00:00,2024-10-12,United Arab Emirates,UCHZArBtk2eda0AVR6HI4qMw,Gaming,0.25,2d,hd,True,1073421,35534,1263,NaT,Morocco,655373060,4710000,False,1746
6,6YMY62tMLUA,2024-10-09 09:00:08+00:00,2024-10-12,United Arab Emirates,UCq-Fj5jknLsUf-MWSy4_brA,Music,3.85,2d,hd,True,56032799,1058450,44767,NaT,India,269373477007,276000000,False,21864
